In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv("../data/Batch_5082026_batch_results (1).csv")

In [38]:
def parse_response(x):
    summary = x["summary"]
    actions = eval(x["log_of_what_they_did"])
    
    data = {"keys": "", "dates": []}
    
    for val in actions:
        data["keys"] += " " + val[1]
        data["dates"].append(val[2])
    data["keys"] = " ".join(data["keys"].split()).strip()
    
    return summary, data

In [39]:
relevant_columns = ["HITId", "original_text","key_strokes", "datetime", "summary"]

responses = pd.DataFrame(columns = relevant_columns)

for i, row in df.iterrows():
    unparsed = eval(row["Answer.taskAnswers"])[0]
    summary, data = parse_response(unparsed)
    
    key_strokes = data["keys"]
    datetime = data["dates"]
    
    temp = pd.DataFrame([[row["HITId"], row["Input.texts"], key_strokes, datetime, summary]], columns = relevant_columns)
    
    responses = pd.concat([responses, temp], axis = 0)
    
    

In [43]:
def locate_paste(x: str) -> bool:
    """
    This function will attempt to use some heuristics to determine
    if the key strokes signify that the text was copied.
    """
    if "Control v" in x:
        return True
    else:
        return False
    
responses["copied"] = responses["key_strokes"].apply(lambda x: locate_paste(x))

In [45]:
responses.groupby("copied")["HITId"].count()

copied
False    20
True     28
Name: HITId, dtype: int64

In [6]:
example = '[{"log_of_what_they_did":"[[\\"key\\",\\" \\",\\"2023-05-29T22:30:57.844Z\\"],[\\"key\\",\\" \\",\\"2023-05-29T22:31:23.058Z\\"],[\\"key\\",\\" \\",\\"2023-05-29T22:31:30.693Z\\"],[\\"key\\",\\"Backspace\\",\\"2023-05-29T22:31:40.132Z\\"],[\\"key\\",\\" \\",\\"2023-05-29T22:31:46.134Z\\"],[\\"key\\",\\"Delete\\",\\"2023-05-29T22:31:48.042Z\\"]]","summary":"Because carbs usually provide over half of calories consumed, the main reason for weight loss on the Atkins Diet is lower overall calorie intake from eating less carbs. Some studies suggest that there are other reasons for weight loss with the Atkins Diet. You may shed pounds because your food choices are limited. And you eat less since the extra protein and fat keep you feeling full longer. Both of these effects also contribute to lower overall calorie intake. The Atkins Diet says that its eating plan can prevent or improve serious health conditions, such as metabolic syndrome, diabetes, high blood pressure and heart disease. In fact, almost any diet that helps you shed excess Some very low carb diets also restrict carbs so much that they cause you not to have enough nutrients or fiber. This can cause such health problems as constipation, diarrhea and nausea. Eating carbs that are high fiber, whole grain and high in nutrients can improve the health profile of programs like the Atkins Diet, though eight can reduce or even reverse risk factors for heart disease and diabetes. \\n\\n"}]'

In [ ]:
def 

In [7]:
ex = eval(example)

[{'log_of_what_they_did': '[["key"," ","2023-05-29T22:30:57.844Z"],["key"," ","2023-05-29T22:31:23.058Z"],["key"," ","2023-05-29T22:31:30.693Z"],["key","Backspace","2023-05-29T22:31:40.132Z"],["key"," ","2023-05-29T22:31:46.134Z"],["key","Delete","2023-05-29T22:31:48.042Z"]]',
  'summary': 'Because carbs usually provide over half of calories consumed, the main reason for weight loss on the Atkins Diet is lower overall calorie intake from eating less carbs. Some studies suggest that there are other reasons for weight loss with the Atkins Diet. You may shed pounds because your food choices are limited. And you eat less since the extra protein and fat keep you feeling full longer. Both of these effects also contribute to lower overall calorie intake. The Atkins Diet says that its eating plan can prevent or improve serious health conditions, such as metabolic syndrome, diabetes, high blood pressure and heart disease. In fact, almost any diet that helps you shed excess Some very low carb di

In [27]:
exx = ex[0]["summary"]

In [11]:
ex2 = ex[0]["log_of_what_they_did"]
ex2 = eval(ex2)

In [24]:
data = {"keys": "", "dates": []}


'Backspace Delete'